In [1]:
from lxml import etree
import pandas as pd
from io import StringIO

In [2]:
doc = etree.parse("fpds_feed_sample_1_org.xml")

In [3]:
xslt_tree = etree.parse("fpds_feed_v3.xsl")
transform = etree.XSLT(xslt_tree)

In [4]:
result = transform(doc)

In [5]:
csv_str =str(result)
csv_str

'mainAccountCode\t4930\t4930\t4930\t4930\t4930\t4930\t4930\t\t0130\t\nsubAccountCode\t\t\t\t\t\t\t\t\t\t\ninitiative\t\t\t\t\t\t\t\t\t\t\ninitiative-description\t\t\t\t\t\t\t\t\t\t\nobligatedAmount\t2501.00\t2501.00\t2501.00\t2501.00\t2501.00\t2501.00\t1961.00\t3454791.72\t559484.40\t323256.60\nreferencedIDVPart8OrPart13\t\t\t\t\t\t\t\t\tPART13\t\nreferencedIDVMultipleOrSingle\tM\tM\tM\tM\tM\tM\tS\tS\tM\tM\nreferencedIDVType\tB\tB\tB\tB\tB\tB\tB\tB\tE\tB\ncontingencyHumanitarianPeacekeepingOperation\tX\tX\tX\tX\tX\tX\tX\t\tX\tX\ncontingencyHumanitarianPeacekeepingOperation-description\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\t\tNOT APPLICABLE\tNOT APPLICABLE\ncontractFinancing\tZ\tZ\tZ\tZ\tZ\tZ\tZ\tZ\tZ\tZ\ncontractFinancing-description\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\npurchaseCardAsPayment

In [6]:
mystr = StringIO(csv_str)

df = pd.read_csv(mystr, sep="\t", header=None, index_col=0)
df.tail(16)

,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,
mainAccountCode,4930,4930,4930,4930,4930,4930,4930,NaN,0130,NaN
subAccountCode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
initiative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
initiative-description,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
obligatedAmount,2501.00,2501.00,2501.00,2501.00,2501.00,2501.00,1961.00,3454791.72,559484.40,323256.60
referencedIDVPart8OrPart13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PART13,NaN
referencedIDVMultipleOrSingle,M,M,M,M,M,M,S,S,M,M
referencedIDVType,B,B,B,B,B,B,B,B,E,B
contingencyHumanitarianPeacekeepingOperation,X,X,X,X,X,X,X,NaN,X,X


In [7]:
df_new = df.transpose(copy=True)
df_new.iloc[:, -16:]

,mainAccountCode,subAccountCode,initiative,initiative-description,obligatedAmount,referencedIDVPart8OrPart13,referencedIDVMultipleOrSingle,referencedIDVType,contingencyHumanitarianPeacekeepingOperation,contingencyHumanitarianPeacekeepingOperation-description,contractFinancing,contractFinancing-description,purchaseCardAsPaymentMethod,purchaseCardAsPaymentMethod-description,numberOfActions
1,4930,NaN,NaN,NaN,2501.00,NaN,M,B,X,NOT APPLICABLE,Z,NOT APPLICABLE,N,NO,1
2,4930,NaN,NaN,NaN,2501.00,NaN,M,B,X,NOT APPLICABLE,Z,NOT APPLICABLE,N,NO,1
3,4930,NaN,NaN,NaN,2501.00,NaN,M,B,X,NOT APPLICABLE,Z,NOT APPLICABLE,N,NO,1
4,4930,NaN,NaN,NaN,2501.00,NaN,M,B,X,NOT APPLICABLE,Z,NOT APPLICABLE,N,NO,1
5,4930,NaN,NaN,NaN,2501.00,NaN,M,B,X,NOT APPLICABLE,Z,NOT APPLICABLE,N,NO,1
6,4930,NaN,NaN,NaN,2501.00,NaN,M,B,X,NOT APPLICABLE,Z,NOT APPLICABLE,N,NO,1
7,4930,NaN,NaN,NaN,1961.00,NaN,S,B,X,NOT APPLICABLE,Z,NOT APPLICABLE,N,NO,1
8,NaN,NaN,NaN,NaN,3454791.72,NaN,S,B,NaN,NaN,Z,NOT APPLICABLE,N,NO,1
9,0130,NaN,NaN,NaN,559484.40,PART13,M,E,X,NOT APPLICABLE,Z,NOT APPLICABLE,N,NO,1
10,NaN,NaN,NaN,NaN,323256.60,NaN,M,B,X,NOT APPLICABLE,Z,NOT APPLICABLE,N,NO,1


In [ ]:
df_new.shape

In [ ]:
# output dataframe to a .csv file
df_new.to_csv("fpds_feed_result.csv", index=False)